In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import json
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import torch

# Model and processor paths
model_dir = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(model_dir).to("cuda")
processor = AutoProcessor.from_pretrained(
    model_dir,
    image_min_pixels=1024,
    image_max_pixels=589824
)
processor.tokenizer.padding_side = 'left'

def patch_tokenizer_and_processor(tokenizer, processor):
    if tokenizer.pad_token is None:
        if tokenizer.eos_token is not None:
            tokenizer.pad_token = tokenizer.eos_token
        else:
            tokenizer.pad_token = "<|endoftext|>"
    if tokenizer.pad_token_id is None:
        if tokenizer.eos_token_id is not None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
        else:
            tokenizer.pad_token_id = 0
    if tokenizer.bos_token_id is None:
        if tokenizer.eos_token_id is not None:
            tokenizer.bos_token_id = tokenizer.eos_token_id
        else:
            tokenizer.bos_token_id = 0
    if tokenizer.eos_token_id is None:
        if hasattr(tokenizer, "eos_token") and tokenizer.eos_token is not None:
            tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
        else:
            tokenizer.eos_token_id = 0
    processor.pad_token = tokenizer.pad_token
    processor.pad_token_id = tokenizer.pad_token_id
    processor.bos_token_id = tokenizer.bos_token_id
    processor.eos_token_id = tokenizer.eos_token_id

patch_tokenizer_and_processor(processor.tokenizer, processor)

def extract_answer(text):
    match = re.search(r"<answer>\s*([A-D])\s*</answer>", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return None

def evaluate(
    test_jsonl,
    model,
    processor,
    device="cuda",
    max_new_tokens=256,
    batch_size=4,
):
    """Evaluate a single test set with optional image visualization."""
    samples = []
    with open(test_jsonl, "r") as f:
        for line in f:
            samples.append(json.loads(line))
    correct = 0
    total = 0

    for i in tqdm(range(0, len(samples), batch_size), desc="Evaluating"):
        batch = samples[i:i+batch_size]
        prompts = [sample["prompt"] for sample in batch]
        def resize_if_needed(img, max_size=768):
            if img.width > max_size or img.height > max_size:
                img.thumbnail((max_size, max_size), Image.LANCZOS)
            return img
        images = [resize_if_needed(Image.open(sample["image_path"]).convert("RGB")) for sample in batch]
        gt_letters = [extract_answer(sample.get("chosen", "")) for sample in batch]
        text_inputs = [
            processor.apply_chat_template(
                [{"role": "user", "content": [
                    {"type": "image"},
                    {"type": "text", "text": prompt}
                ]}],
                tokenize=False,
                add_generation_prompt=True
            )
            for prompt in prompts
        ]
        inputs = processor(text=text_inputs, images=images, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
        input_len = inputs['input_ids'].shape[1]
        decoded = processor.batch_decode(generated_ids[:, input_len:], skip_special_tokens=True)
        for idx, (pred, gt, prompt, image_path, img) in enumerate(zip(decoded, gt_letters, prompts, [sample["image_path"] for sample in batch], images)):
            pred_letter = extract_answer(pred)
            if pred_letter and gt and pred_letter == gt:
                correct += 1
            total += 1

        del inputs, generated_ids
        torch.cuda.empty_cache()
    accuracy = correct / total if total > 0 else 0
    print(f"\nAccuracy: {accuracy:.4f} ({correct}/{total})")

test_sets = {
    "MRI (in-domain)": "/home/vishnu/data/llamafactory_env/LLaMA-Factory/test_sets/test_mri_dpo.jsonl",
    "CT (OOD)": "/home/vishnu/data/llamafactory_env/LLaMA-Factory/test_sets/test_ct_dpo.jsonl",
    "X-ray (OOD)": "/home/vishnu/data/llamafactory_env/LLaMA-Factory/test_sets/test_xray_dpo.jsonl"
}

/mnt/vdb/vishnu/llamafactory_env/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Loading checkpoint shards:   0%|                                                                                                                                                      | 0/2 [00:00<?, ?it/s]


Loading checkpoint shards:  50%|███████████████████████████████████████████████████████████████████████                                                                       | 1/2 [00:01<00:01,  1.22s/it]


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.78it/s]


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.51it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [2]:
print("\nEvaluating on MRI (in-domain)...")
evaluate(
    test_sets["MRI (in-domain)"],
    model,
    processor,
    batch_size=4
)


Evaluating on MRI (in-domain)...



Evaluating:   0%|                                                                                                                                                                    | 0/38 [00:00<?, ?it/s]


Evaluating:   3%|████                                                                                                                                                        | 1/38 [00:05<03:10,  5.16s/it]


Evaluating:   5%|████████▏                                                                                                                                                   | 2/38 [00:09<02:45,  4.60s/it]


Evaluating:   8%|████████████▎                                                                                                                                               | 3/38 [00:13<02:28,  4.25s/it]

...


Evaluating:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊        | 36/38 [02:25<00:08,  4.14s/it]


Evaluating:  97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉    | 37/38 [02:29<00:04,  4.30s/it]


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [02:32<00:00,  3.75s/it]


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [02:32<00:00,  4.01s/it]


Accuracy: 0.3200 (48/150)


In [3]:
print("\nEvaluating on CT (OOD)...")
evaluate(
    test_sets["CT (OOD)"],
    model,
    processor,
    batch_size=4
)


Evaluating on CT (OOD)...



Evaluating:   0%|                                                                                                                                                                    | 0/38 [00:00<?, ?it/s]


Evaluating:   3%|████                                                                                                                                                        | 1/38 [00:04<02:43,  4.42s/it]


Evaluating:   5%|████████▏                                                                                                                                                   | 2/38 [00:09<02:45,  4.59s/it]


Evaluating:   8%|████████████▎                                                                                                                                               | 3/38 [00:12<02:18,  3.97s/it]

...


Evaluating:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊        | 36/38 [02:20<00:07,  3.92s/it]


Evaluating:  97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉    | 37/38 [02:23<00:03,  3.52s/it]


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [02:26<00:00,  3.46s/it]


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [02:26<00:00,  3.86s/it]


Accuracy: 0.3467 (52/150)


In [ ]:
print("\nEvaluating on X-ray (OOD)...")
evaluate(
    test_sets["X-ray (OOD)"],
    model,
    processor,
    batch_size=4
)


Evaluating on X-ray (OOD)...



Evaluating:   0%|                                                                                                                                                                    | 0/38 [00:00<?, ?it/s]


Evaluating:   3%|████                                                                                                                                                        | 1/38 [00:06<03:50,  6.24s/it]


Evaluating:   5%|████████▏                                                                                                                                                   | 2/38 [00:11<03:30,  5.84s/it]


Evaluating:   8%|████████████▎                                                                                                                                               | 3/38 [00:18<03:33,  6.10s/it]

...


Evaluating:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊        | 36/38 [03:39<00:11,  5.91s/it]


Evaluating:  97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉    | 37/38 [03:43<00:05,  5.35s/it]


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [03:47<00:00,  4.85s/it]


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [03:47<00:00,  5.98s/it]


Accuracy: 0.3733 (56/150)
